In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [3]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

In [4]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(indep)

RANDOM_FOREST

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param_grid={'criterion':['log_loss', 'entropy', 'gini'],
              'max_features': [None,'sqrt','log2'],
              'n_estimators':[10,100]} 
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['log_loss', 'entropy', 'gini'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [6]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)


[[150   0]
 [  0 249]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399

The f1_macro value for the best parameters {'criterion': 'log_loss', 'max_features': 'log2', 'n_estimators': 10}: 1.0


In [7]:
from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

The roc_score: 1.0


In [8]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.067004,0.028794,0.039210,0.042820,log_loss,None,10,"{'criterion': 'log_loss', 'max_features': None...",0.840046,0.987456,0.936147,0.987539,1.00000,0.950238,0.059319,15
1,0.655251,0.058177,0.032894,0.018500,log_loss,None,100,"{'criterion': 'log_loss', 'max_features': None...",0.901042,0.975000,0.936147,0.987539,1.00000,0.959946,0.036410,13
2,0.085952,0.023634,0.015527,0.014938,log_loss,sqrt,10,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.901354,1.000000,0.975000,0.987539,1.00000,0.972779,0.036895,9
3,0.344698,0.061706,0.026425,0.022552,log_loss,sqrt,100,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.938022,0.987456,0.974814,0.987539,1.00000,0.977566,0.021316,8
4,0.037994,0.017308,0.013005,0.011234,log_loss,log2,10,"{'criterion': 'log_loss', 'max_features': 'log...",0.987539,1.000000,0.974814,0.987539,1.00000,0.989979,0.009410,1
5,0.563698,0.127224,0.037587,0.016629,log_loss,log2,100,"{'criterion': 'log_loss', 'max_features': 'log...",0.950296,0.987456,0.987456,0.987539,1.00000,0.982549,0.016839,3
6,0.093165,0.069896,0.008572,0.001640,entropy,None,10,"{'criterion': 'entropy', 'max_features': None,...",0.790000,0.987456,0.936147,0.975148,1.00000,0.937750,0.076912,17
7,0.520598,0.126346,0.025296,0.012609,entropy,None,100,"{'criterion': 'entropy', 'max_features': None,...",0.876918,0.987539,0.936147,0.987539,1.00000,0.957629,0.045957,14
8,0.061314,0.015586,0.013234,0.013236,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.950521,1.000000,0.974814,0.987539,1.00000,0.982575,0.018545,2
9,0.426468,0.099737,0.027719,0.016736,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.938022,1.000000,0.974814,0.987539,1.00000,0.980075,0.023004,4
